In [ ]:
# !nvidia-smi

Thu Apr  1 08:25:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# %cd /content/drive/MyDrive/S2 MIK/Tesis Raditya Nurfadillah/CARL
# %cd /content/drive/MyDrive/Tesis Raditya Nurfadillah/CARL

/content/drive/MyDrive/S2 MIK/Tesis Raditya Nurfadillah/CARL


In [ ]:
!ls

 aiv
 aiv.tar
'CARL BERT.ipynb'
'CARL FastText 2.ipynb'
'CARL FastText Document Average.ipynb'
'CARL FastText.ipynb'
'CARL FastText (Surprise + Review-based) 2.ipynb'
'CARL FastText (Surprise + Review-based) High Computing Power 2.ipynb'
'CARL FastText (Surprise + Review-based) High Computing Power 3.ipynb'
'CARL FastText (Surprise + Review-based) High Computing Power.ipynb'
'CARL FastText (Surprise + Review-based).ipynb'
'CARL FemaleDaily.ipynb'
'CARL High Computing Power.ipynb'
 CARL.ipynb
'CARL Tensorflow 2 - 2.ipynb'
'CARL Tensorflow 2 - 3.ipynb'
'CARL Tensorflow 2.ipynb'
 cc.en.300.bin
 cc.id.300.bin
'Copy of CARL.ipynb'
'Copy of Copy of CARL High Computing Power.ipynb'
'Copy of Salinan dari P100.ipynb'
 Dataset
'Download CARL Dataset.ipynb'
'EDA Amazon + FemaleDaily.ipynb'
 EDA.ipynb
 Model
'prediction beauty.csv'
'Prediction CARL'
'prediction FemaleDaily.csv'
'prediction FemaleDaily epoch 0.csv'
'prediction FemaleDaily epoch 100.csv'
'prediction FemaleDaily epoch 150.csv'
'predi

In [ ]:
# %tensorflow_version 1.x

In [ ]:
import tensorflow
print(tensorflow.__version__)

2.4.1


In [ ]:
import tensorflow as tf

# Set a seed value
seed_value = 24
tf.random.set_seed(seed_value)

In [ ]:
!pip install tensorflow-addons

# Extract Data

In [ ]:
import numpy as np
import scipy.sparse as sp

class Dataset(object):
    "'extract dataset from file'"

    def __init__(self, max_length, path, word_id_path):
        self.word_id_dict = self.load_word_dict(path + word_id_path)
        print("wordId_dict finished")
        self.userReview_dict = self.load_reviews(max_length, len(self.word_id_dict), path + "UserReviews.out")
        self.itemReview_dict = self.load_reviews(max_length, len(self.word_id_dict), path + "ItemReviews.out")
        print("load reviews finished")
        self.num_users, self.num_items = len(self.userReview_dict), len(self.itemReview_dict)
        print(self.num_users, self.num_items)
        self.trainMtrx = self.load_ratingFile_as_mtrx(path + "TrainInteraction.out")
        self.valRatings = self.load_ratingFile_as_list(path + "ValInteraction.out")
        self.testRatings = self.load_ratingFile_as_list(path + "TestInteraction.out")

    def load_word_dict(self, path):
        wordId_dict = {}

        with open(path, "r") as f:
            line = f.readline().replace("\n", "")
            while line != None and line != "":
                arr = line.split("\t")
                wordId_dict[arr[0]] = int(arr[1])
                line = f.readline().replace("\n", "")

        return wordId_dict

    def load_reviews(self, max_doc_length, padding_word_id, path):
        entity_review_dict = {}

        with open(path, "r") as f:
            line = f.readline().replace("\n", "")
            while line != None and line != "":
                review = []
                arr = line.split("\t")
                entity = int(arr[0])
                word_list = arr[1].split(" ")

                for i in range(len(word_list)):
                    # if (word_list[i] == "" or word_list[i] == None or (not self.word_id_dict.has_key(word_list[i]))):
                    if (word_list[i] == "" or word_list[i] == None or (not word_list[i] in self.word_id_dict)):
                        continue
                    review.append(self.word_id_dict.get(word_list[i]))
                    if (len(review) >= max_doc_length):
                        break
                if (len(review) < max_doc_length):
                    review = self.padding_word(max_doc_length, padding_word_id, review)
                entity_review_dict[entity] = review
                line = f.readline().replace("\n", "")
        return entity_review_dict

    def padding_word(self, max_size, max_word_idx, review):
        review.extend([max_word_idx]*(max_size - len(review)))
        return review

    def load_ratingFile_as_mtrx(self, file_path):
        mtrx = sp.dok_matrix((self.num_users, self.num_items), dtype=np.float32)
        with open(file_path, "r") as f:
            line = f.readline()
            line = line.strip()
            while line != None and line != "":
                arr = line.split("\t")
                user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
                if (rating > 0):
                    mtrx[user, item] = rating
                line = f.readline()

        return mtrx

    def load_ratingFile_as_list(self, file_path):
        rateList = []

        with open(file_path, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item = int(arr[0]), int(arr[1])
                rate = float(arr[2])
                rateList.append([user, item, rate])
                line = f.readline()

        return rateList

# Test Data

In [ ]:
import math

def get_test_list(batch_size, test_rating, user_reviews, item_reviews):
    user_test_batchs, item_test_batchs, user_input_test_batchs, item_input_test_batchs, rating_input_test_batchs = [], [], [], [], []
    for count in range(int(math.ceil(len(test_rating) / float(batch_size)))):
        user_test, item_test, user_input_test, item_input_test, rating_input_test = [], [], [], [], []
        for idx in range(batch_size):
            index = (count * batch_size + idx)
            if (index >= len(test_rating)):
                break
            rating = test_rating[index]
            user_test.append(rating[0])
            item_test.append(rating[1])
            user_input_test.append(user_reviews.get(rating[0]))
            item_input_test.append(item_reviews.get(rating[1]))
            rating_input_test.append([rating[2]])
        user_test_batchs.append(user_test)
        item_test_batchs.append(item_test)
        user_input_test_batchs.append(user_input_test)
        item_input_test_batchs.append(item_input_test)
        rating_input_test_batchs.append(rating_input_test)
        #print count, len(item_input_test_batchs[count])
    return user_test_batchs, item_test_batchs, user_input_test_batchs, item_input_test_batchs, rating_input_test_batchs

# FastText

In [ ]:
# Download dan unzip dataset
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz

In [ ]:
!gunzip cc.en.300.bin.gz

In [ ]:
!pip install fasttext

In [ ]:
import fasttext

fasttext_model = fasttext.load_model("cc.en.300.bin")

# Interaction

In [ ]:
def get_train_instance(train):
    user_input, item_input, rates = [], [], []

    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        rates.append(train[u,i])
    return user_input, item_input, rates

def get_train_instance_batch_change(count, batch_size, user_input, item_input, ratings, user_reviews, item_reviews):
    users_batch, items_batch, user_input_batch, item_input_batch, labels_batch = [], [], [], [], []

    for idx in range(batch_size):
        index = (count*batch_size + idx) % len(user_input)
        users_batch.append(user_input[index])
        items_batch.append(item_input[index])
        user_input_batch.append(user_reviews.get(user_input[index]))
        item_input_batch.append(item_reviews.get(item_input[index]))
        labels_batch.append([ratings[index]])

    return users_batch, items_batch, user_input_batch, item_input_batch, labels_batch

In [ ]:
import numpy as np
from time import time
import math, os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
word_latent_dim = 300
latent_dim = 30
max_doc_length = 300
windows = 3
v_dim = 50
learning_rate = 0.001
lambda_1 = 0.05
batch_size = 100
epochs = 180

# loading data
firTime = time()
dataset_path = "Dataset/"
category = "patio"
path = dataset_path + category + "/"
dataSet = Dataset(max_doc_length, path,"WordDict.out")
word_dict, user_reviews, item_reviews, train, testRatings = dataSet.word_id_dict, dataSet.userReview_dict, dataSet.itemReview_dict, dataSet.trainMtrx, dataSet.testRatings
secTime = time()

num_users, num_items = train.shape
print("load data: %.3fs" % (secTime - firTime))
print(num_users, num_items)

# get train instances
user_input, item_input, rateings = get_train_instance(train)

# get test instances
user_tests, item_tests, user_input_test, item_input_test, rating_input_test = get_test_list(200, testRatings, user_reviews, item_reviews)

wordId_dict finished
load reviews finished
35598 18357
load data: 32.517s
35598 18357


In [ ]:
class My_Model():
  def __init__(self):
    self.user_entity_embedding = tf.Variable(tf.random.normal([num_users, latent_dim], mean=0, stddev=0.02), name="user_entity_embeddings")
    self.item_entity_embedding = tf.Variable(tf.random.normal([num_items, latent_dim], mean=0, stddev=0.02), name="item_entity_embeddings")
    self.w_entity_0 = tf.Variable(tf.zeros(1), name="entity_w_0")
    self.w_entity_1 = tf.Variable(tf.random.truncated_normal([1, latent_dim*3], stddev=0.3), name="entity_w_1")
    self.v_entity = tf.Variable(tf.random.truncated_normal([latent_dim*3, v_dim], stddev=0.3), name="entity_v")

  def run(self, users, items):
    user_entity_embeds = tf.nn.embedding_lookup(self.user_entity_embedding, users)
    item_entity_embeds = tf.nn.embedding_lookup(self.item_entity_embedding, items)

    entity_embeds_sum = tf.concat([tf.multiply(user_entity_embeds, item_entity_embeds), user_entity_embeds, item_entity_embeds],1)

    J_e_1 = self.w_entity_0 + tf.linalg.matmul(entity_embeds_sum, self.w_entity_1, transpose_b=True)

    entity_embeds_sum_1 = tf.expand_dims(entity_embeds_sum, -1)
    entity_embeds_sum_2 = tf.expand_dims(entity_embeds_sum, 1)
    J_e_2 = tf.reduce_sum(
            tf.reduce_sum(tf.multiply(tf.matmul(entity_embeds_sum_1, entity_embeds_sum_2), tf.matmul(self.v_entity, self.v_entity, transpose_b=True)),
                          2), 1, keepdims=True)
    J_e_3 = tf.linalg.trace(tf.multiply(tf.matmul(entity_embeds_sum_1, entity_embeds_sum_2), tf.matmul(self.v_entity, self.v_entity, transpose_b=True)))
    J_e_total = (J_e_1 + 0.5 * (J_e_2 - J_e_3))
    outputs = J_e_total
    return outputs

  def trainable_variables(self):
    return [self.user_entity_embedding, self.item_entity_embedding, self.w_entity_0, self.w_entity_1, self.v_entity]
    # self.weights + self.bias

  def loss_function(self,y_pred,y_true):
    return tf.reduce_mean(tf.math.squared_difference(y_pred, y_true)) + lambda_1 * (tf.nn.l2_loss(self.user_entity_embedding) + tf.nn.l2_loss(self.item_entity_embedding) + tf.nn.l2_loss(self.v_entity))

In [ ]:
my_model = My_Model()

In [ ]:
import tensorflow_addons as tfa

# lr = 1e-3
# wd = 1e-4
# optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

optimizer = tf.optimizers.Adam(learning_rate=0.001)

In [ ]:
def train_step( model, users, items , y_true, epoch):
    
  with tf.GradientTape() as tape:
        
    # Get the predictions
    preds = model.run(users, items)
        
    # Calc the loss
    current_loss = model.loss_function(preds,y_true)
    
    # Get the gradients
    grads = tape.gradient(current_loss, model.trainable_variables())
    
    # Update the weights
    optimizer.apply_gradients(zip(grads, model.trainable_variables()))

    return current_loss

In [ ]:
def eval_model(model, user_test, item_test, rate_tests, rmses, maes):

    # predicts = sess.run(predict_rating, feed_dict={users: user_test, items: item_test})
    predicts = model.run(user_test, item_test)
    row, col = predicts.shape
    for r in range(row):
        rmses.append(pow((predicts[r, 0] - rate_tests[r][0]), 2))
        maes.append(abs(predicts[r, 0] - rate_tests[r][0]))
    return rmses, maes

In [ ]:
for epoch in range(epochs):
  t = time()
  loss_total = 0.0
  count = 0.0          
  for i in range(int(math.ceil(len(user_input) / float(batch_size)))):
    user_batch, item_batch, user_input_batch, item_input_batch, rates_batch = get_train_instance_batch_change(i, batch_size,user_input,
                                                                                                    item_input, rateings,
                                                                                                    user_reviews,item_reviews)

    loss_val = train_step(my_model, user_batch, item_batch, rates_batch, epoch)
    loss_total += loss_val
    count += 1.0

  t1 = time()
  mses, maes = [], []
  for i in range(len(user_input_test)):
      mses, maes = eval_model(my_model, user_tests[i], item_tests[i], rating_input_test[i], mses, maes)
  mse = np.array(mses).mean()
  mae = np.array(maes).mean()
  t2 = time()

  print("epoch%d train time: %.3fs test time: %.3f  loss = %.3f  mse = %.3f  mae = %.3f"%(epoch, (t1 - t), (t2 - t1), loss_total/count, mse, mae))

epoch0 train time: 24.762s test time: 21.665  loss = 13.542  mse = 4.677  mae = 1.973
epoch1 train time: 23.922s test time: 21.531  loss = 4.718  mse = 1.481  mae = 1.036
epoch2 train time: 24.148s test time: 21.664  loss = 2.035  mse = 1.010  mae = 0.780
epoch3 train time: 24.071s test time: 21.571  loss = 1.499  mse = 0.992  mae = 0.771
epoch4 train time: 23.671s test time: 21.466  loss = 1.410  mse = 0.996  mae = 0.775


# Review

In [ ]:
# encoding: utf-8
# import tensorflow as tf
import pandas as pd
import numpy as np
from time import time
import math, os


def ini_word_embed(num_words, latent_dim):
    word_embeds = np.random.rand(num_words, latent_dim)
    return word_embeds

def word2vec_word_embed(num_words, latent_dim, path, word_id_dict):
    word2vect_embed_mtrx = np.zeros((num_words, latent_dim))
    with open(path, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            row_id = word_id_dict.get(arr[0])
            vect = arr[1].strip().split(" ")
            for i in range(len(vect)):
                word2vect_embed_mtrx[row_id, i] = float(vect[i])
            line = f.readline()

    return word2vect_embed_mtrx


def fasttext_word_embed(num_words, latent_dim, pretrained_model, word_id_dict):
    fasttext_embed_mtrx = np.zeros((num_words, latent_dim))
    for word in word_id_dict:
      fasttext_embed_mtrx[word_id_dict[word]] = pretrained_model[word]
      
    return fasttext_embed_mtrx

def get_train_instance(train):
    user_input, item_input, rates = [], [], []

    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        rates.append(train[u,i])
    return user_input, item_input, rates

def get_train_instance_batch_change(count, batch_size, user_input, item_input, ratings, user_reviews, item_reviews):
    users_batch, items_batch, user_input_batch, item_input_batch, labels_batch = [], [], [], [], []

    for idx in range(batch_size):
        index = (count*batch_size + idx) % len(user_input)
        users_batch.append(user_input[index])
        items_batch.append(item_input[index])
        user_input_batch.append(user_reviews.get(user_input[index]))
        item_input_batch.append(item_reviews.get(item_input[index]))
        labels_batch.append([ratings[index]])

    return users_batch, items_batch, user_input_batch, item_input_batch, labels_batch

In [ ]:
import numpy as np
from time import time
import math, os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
word_latent_dim = 300
latent_dim = 30
max_doc_length = 300
# windows = 3
num_filters = 50
window_size = 3
v_dim = 50
learning_rate = 0.001
lambda_1 = 0.01
# batch_size = 100
drop_out = 0.8
batch_size = 200
epochs = 180


# loading data
firTime = time()
dataset_path = "Dataset/"
category = "patio"
path = dataset_path + category + "/"
dataSet = Dataset(max_doc_length, path,"WordDict.out")
word_dict, user_reviews, item_reviews, train, valRatings, testRatings = dataSet.word_id_dict, dataSet.userReview_dict, dataSet.itemReview_dict, dataSet.trainMtrx, dataSet.valRatings, dataSet.testRatings
secTime = time()


num_users, num_items = train.shape
print("load data: %.3fs" % (secTime - firTime))
print(num_users, num_items)

# load word embeddings
word_embedding_mtrx = fasttext_word_embed(len(word_dict), word_latent_dim, fasttext_model, word_dict)
print("shape", word_embedding_mtrx.shape)
print(word_embedding_mtrx)

# get train instances
user_input, item_input, rateings = get_train_instance(train)

# get test instances
user_vals, item_vals, user_input_val, item_input_val, rating_input_val = get_test_list(200, valRatings, user_reviews, item_reviews)
user_tests, item_tests, user_input_test, item_input_test, rating_input_test = get_test_list(200, testRatings, user_reviews, item_reviews)

wordId_dict finished
load reviews finished
22363 12101
load data: 15.869s
22363 12101
shape (189590, 300)
[[ 0.34721792 -0.23254557  0.08277421 ...  0.5184263  -0.15084337
  -0.22017725]
 [-0.25729463 -0.24384813 -0.29711977 ... -0.09914862 -0.69585395
   0.21937728]
 [-0.18768255 -0.40687621  0.11908793 ...  0.28632459 -0.0473632
  -0.12526543]
 ...
 [ 0.0032126  -0.00386322  0.00366708 ... -0.00490303 -0.00736631
  -0.00388915]
 [-0.00172517  0.01667362  0.00282218 ... -0.00159003 -0.00430298
  -0.00606333]
 [-0.00323265  0.00319192  0.00483765 ...  0.00491108  0.00687607
  -0.00377376]]


In [ ]:
def cnn_model_average(filters, user_reviews_representation_expnd, item_reviews_representation_expnd, W_u, W_i, W_u_1, W_i_1, rand_matrix):
    #convolution layer
    convU = tf.nn.conv2d(user_reviews_representation_expnd, W_u, strides=[1, 1, word_latent_dim, 1], padding='SAME')
    convI = tf.nn.conv2d(item_reviews_representation_expnd, W_i, strides=[1, 1, word_latent_dim, 1], padding='SAME')

    hU = tf.nn.relu(tf.squeeze(convU, 2))
    hI = tf.nn.relu(tf.squeeze(convI, 2))

    #attentive layer
    sec_dim = int(hU.get_shape()[1])
    tmphU = tf.reshape(hU, [-1, filters])
    hU_mul_rand = tf.reshape(tf.matmul(tmphU, rand_matrix), [-1, sec_dim, filters])
    f = tf.matmul(hU_mul_rand, hI, transpose_b=True)
    f = tf.expand_dims(f, -1)
    att1 = tf.tanh(f)

    pool_user = tf.reduce_mean(att1, 2)
    pool_item = tf.reduce_mean(att1, 1)

    user_flat = tf.squeeze(pool_user, -1)
    item_flat = tf.squeeze(pool_item, -1)
    weight_user = tf.nn.softmax(user_flat)
    weight_item = tf.nn.softmax(item_flat)

    weight_user_exp = tf.expand_dims(weight_user, -1)
    weight_item_exp = tf.expand_dims(weight_item, -1)

    hU = tf.expand_dims(hU * weight_user_exp, -1)
    hI = tf.expand_dims(hI * weight_item_exp, -1)

    #abstracting layer
    hU_1 = tf.nn.conv2d(hU, W_u_1, strides=[1, 1, 1, 1], padding='VALID')
    hI_1 = tf.nn.conv2d(hI, W_i_1, strides=[1, 1, 1, 1], padding='VALID')

    sec_dim = hU_1.get_shape()[1]

    oU = tf.nn.avg_pool(hU_1, ksize=[1, sec_dim, 1, 1], strides=[1, 1, 1, 1],
        padding='VALID')
    oI = tf.nn.avg_pool(hI_1, ksize=[1, sec_dim, 1, 1], strides=[1, 1, 1, 1],
        padding='VALID')

    att_user = tf.squeeze(oU)
    att_item = tf.squeeze(oI)
    #print "attention", att_user.get_shape(), att_item.get_shape()

    return att_user, att_item

In [ ]:
class Review_Model():
  def __init__(self, word_embedding_mtrx, dropout_rate):
    self.text_embedding = tf.Variable(word_embedding_mtrx, dtype=tf.float32, name="review_text_embeds")
    self.padding_embedding = tf.Variable(np.zeros([1, word_latent_dim]), dtype=tf.float32)
    self.text_mask = tf.constant([1.0] * self.text_embedding.get_shape()[0] + [0.0])
    self.user_bias = tf.Variable(tf.random.normal([num_users, 1], mean=0, stddev=0.02), name="review_user_bias")
    self.item_bias = tf.Variable(tf.random.normal([num_items, 1], mean=0, stddev=0.02), name="review_item_bias")
    self.word_embeddings = tf.concat([self.text_embedding, self.padding_embedding], 0) * tf.expand_dims(self.text_mask, -1)

    self.W_u = tf.Variable(
        tf.random.truncated_normal([window_size, word_latent_dim, 1, num_filters], stddev=0.3), name="review_W_u")
    self.W_i = tf.Variable(
        tf.random.truncated_normal([window_size, word_latent_dim, 1, num_filters], stddev=0.3), name="review_W_i")
    self.W_u_1 = tf.Variable(
        tf.random.truncated_normal([window_size, num_filters, 1, num_filters], stddev=0.3), name="review_W_u_1")
    self.W_i_1 = tf.Variable(
        tf.random.truncated_normal([window_size, num_filters, 1, num_filters], stddev=0.3), name="review_W_i_1")
    # b = tf.Variable(tf.constant(0.1, shape=[self.num_filters]))
    self.rand_matrix = tf.Variable(tf.random.truncated_normal([num_filters, num_filters], stddev=0.3), name="review_rand_matrix")

    #shared mlp layer
    self.W_mlp = tf.nn.dropout(tf.Variable(tf.random.normal([num_filters, latent_dim], mean=0, stddev=0.02), name="review_W_mlp"), dropout_rate)
    self.b_mlp = tf.Variable(tf.constant(0.1, shape=[latent_dim]), name="review_b_mlp")

    #FM predictor
    self.w_0 = tf.Variable(tf.zeros(1), name="review_w_0")
    self.w_1 = tf.Variable(tf.random.truncated_normal([1, latent_dim * 3], stddev=0.3), name="review_w_1")
    self.v = tf.Variable(tf.random.truncated_normal([latent_dim * 3, v_dim], stddev=0.3), name="review_v")

    self.user_bs = None
    self.item_bs = None
    
  def run(self, users, items, users_inputs, items_inputs):
    self.user_bs = tf.nn.embedding_lookup(self.user_bias, users)
    self.item_bs = tf.nn.embedding_lookup(self.item_bias, items)

    user_reviews_representation = tf.nn.embedding_lookup(self.word_embeddings, users_inputs)
    user_reviews_representation_expnd = tf.expand_dims(user_reviews_representation, -1)
    item_reviews_representation = tf.nn.embedding_lookup(self.word_embeddings, items_inputs)
    item_reviews_representation_expnd = tf.expand_dims(item_reviews_representation, -1)

    user_embeds, item_embeds = cnn_model_average(num_filters, user_reviews_representation_expnd, item_reviews_representation_expnd, self.W_u, self.W_i, self.W_u_1, self.W_i_1, self.rand_matrix)
    
    user_embeds = tf.nn.relu(tf.matmul(user_embeds, self.W_mlp) + self.b_mlp)
    item_embeds = tf.nn.relu(tf.matmul(item_embeds, self.W_mlp) + self.b_mlp)

    embeds_sum = tf.concat([tf.multiply(user_embeds, item_embeds), user_embeds, item_embeds], 1, name="concat_embed")

    J_1 = self.w_0 + tf.matmul(embeds_sum, self.w_1, transpose_b=True)

    embeds_sum_1 = tf.expand_dims(embeds_sum, -1)
    embeds_sum_2 = tf.expand_dims(embeds_sum, 1)

    J_2 = tf.reduce_sum(
        tf.reduce_sum(tf.multiply(tf.matmul(embeds_sum_1, embeds_sum_2), tf.matmul(self.v, self.v, transpose_b=True)),
                      2), 1, keepdims=True)
    J_3 = tf.linalg.trace(tf.multiply(tf.matmul(embeds_sum_1, embeds_sum_2), tf.matmul(self.v, self.v, transpose_b=True)))
    J_total = (J_1 + 0.5 * (J_2 - J_3))
    outputs = J_total + self.user_bs + self.item_bs
    return outputs

  def trainable_variables(self):
    return [self.user_bias, self.item_bias, self.W_u, self.W_i, self.W_u_1, self.W_i_1, self.w_0, self.w_1, self.v]
    
  def loss_function(self,y_pred,y_true):
    return tf.reduce_mean(tf.math.squared_difference(y_pred, y_true)) + lambda_1 * (tf.nn.l2_loss(self.W_i_1) + tf.nn.l2_loss(self.W_u_1) + tf.nn.l2_loss(self.W_u) + tf.nn.l2_loss(self.W_i) + tf.nn.l2_loss(self.v) + tf.nn.l2_loss(self.rand_matrix) + tf.nn.l2_loss(self.user_bs) + tf.nn.l2_loss(self.item_bs))

In [ ]:
my_review_model = Review_Model(word_embedding_mtrx,drop_out)

In [ ]:
optimizer = tf.optimizers.Adam(learning_rate=0.001)

In [ ]:
def train_step( model, users, items , users_inputs, items_inputs, y_true, epoch):
    
  with tf.GradientTape() as tape:
        
    # Get the predictions
    preds = model.run(users, items, users_inputs, items_inputs)
        
    # Calc the loss
    current_loss = model.loss_function(preds,y_true)
    
    # Get the gradients
    grads = tape.gradient(current_loss, model.trainable_variables())
    
    # Update the weights
    optimizer.apply_gradients(zip(grads, model.trainable_variables()))

    return current_loss

In [ ]:
def eval_model(model, user_test, item_test, users_inputs_test, items_inputs_test, rate_tests, rmses, maes):

    # predicts = sess.run(predict_rating, feed_dict={users: user_test, items: item_test})
    predicts = model.run(user_test, item_test, users_inputs_test, items_inputs_test)
    row, col = predicts.shape
    for r in range(row):
        rmses.append(pow((predicts[r, 0] - rate_tests[r][0]), 2))
        maes.append(abs(predicts[r, 0] - rate_tests[r][0]))
    return rmses, maes

In [ ]:
for epoch in range(epochs):
  t = time()
  loss_total = 0.0
  count = 0.0
          
  for i in range(int(math.ceil(len(user_input) / float(batch_size)))):
    user_batch, item_batch, user_input_batch, item_input_batch, rates_batch = get_train_instance_batch_change(i, batch_size,user_input,
                                                                                                    item_input, rateings,
                                                                                                    user_reviews,item_reviews)

    loss_val = train_step(my_review_model, user_batch, item_batch, user_input_batch, item_input_batch, rates_batch, epoch)
    loss_total += loss_val
    count += 1.0

  t1 = time()
  mses, maes = [], []
  for i in range(len(user_input_test)):
      mses, maes = eval_model(my_review_model, user_tests[i], item_tests[i], user_input_test[i], item_input_test[i], rating_input_test[i], mses, maes)
  mse = np.array(mses).mean()
  mae = np.array(maes).mean()
  t2 = time()

  print("epoch%d train time: %.3fs test time: %.3f  loss = %.3f  mse = %.3f  mae = %.3f"%(epoch, (t1 - t), (t2 - t1), loss_total/count, mse, mae))

epoch0 train time: 98.271s test time: 30.998  loss = 10.932  mse = 1.706  mae = 0.837
epoch1 train time: 66.730s test time: 31.522  loss = 2.761  mse = 1.605  mae = 0.832
epoch2 train time: 66.165s test time: 30.670  loss = 2.419  mse = 1.542  mae = 0.826
epoch3 train time: 66.915s test time: 32.188  loss = 2.241  mse = 1.500  mae = 0.818
epoch4 train time: 67.427s test time: 30.833  loss = 2.147  mse = 1.475  mae = 0.810
epoch5 train time: 66.616s test time: 30.950  loss = 2.094  mse = 1.464  mae = 0.803
epoch6 train time: 66.074s test time: 30.795  loss = 2.062  mse = 1.461  mae = 0.796
epoch7 train time: 65.754s test time: 30.589  loss = 2.039  mse = 1.463  mae = 0.793
epoch8 train time: 65.510s test time: 30.657  loss = 2.020  mse = 1.463  mae = 0.792
epoch9 train time: 66.419s test time: 31.268  loss = 2.004  mse = 1.466  mae = 0.792
epoch10 train time: 65.930s test time: 31.180  loss = 1.991  mse = 1.433  mae = 0.789
epoch11 train time: 66.238s test time: 30.899  loss = 1.979  ms